## フレームワークの紹介

- tensorflow
- keras
- pytorch
- chainer

などが有名な deep learning フレームワークです。

deep learning は微分を計算する必要がありますが、その計算のサポートなどをやってくれるのがこれらのフレームワークです。

大きく分けると 2 種類に分類できます。

### Define by Run

chainer や pytorch はこちらに属します。
これらは、実際に計算をしながら計算グラフを構築し、自動微分を行うフレームワークです。
言葉で説明すると難しいですが、ちょっと例を動かすとわかりやすいと思うので例を書きます。

In [1]:
!pip install torch

import torch

x = torch.tensor([3.0], requires_grad=True)
y = torch.tensor([2.0])

z = x * y
print(z) # この時点で 3.0 * 2.0 = 6.0 が計算されていることがわかる。=> 計算しながら微分に必要な計算グラフを構築している

z.backward() # 微分を計算する。これを計算するには、z がどんな計算式で作られているか覚えている必要がある = 計算グラフを構築する必要がある
print(x.grad) # dz/dx 

    100% |████████████████████████████████| 519.5MB 18kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58aae000 @  0x7f4dc2d2c1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
tensor([6.], grad_fn=<ThMulBackward>)
tensor([2.])


PyTorch でのニューラルネットワークの簡単な例を↓にかきます。

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

`forward` メソッドがネットワークの構造を決定しています。  
`x` という入力を、いくつかの演算、活性化関数に通して、最終的な出力を得ています。


### Define and Run

tensorflow などは define and run と呼ばれるフレームワークです。

前もって計算グラフを構築し、できあがった計算グラフを評価する、という方法で動きます。
これも例を見るとわかりやすいので例を出します。

In [3]:
import tensorflow as tf

x = tf.constant([3.0])
y = tf.constant([2.0])

z = x * y
print(z) # この時点ではまだ値は計算されない

tf.InteractiveSession().run(z) # 構築された計算グラフを評価

x = tf.placeholder(tf.float32, shape=(1,)) # （計算グラフ上の）変数を定義する。評価するときにこの値を実データで置き換えられる
y = tf.constant([2.0])
z = x * y
print(z) # まだ計算されない

tf.InteractiveSession().run(z, feed_dict={x: [3.0]}) # x という変数を [3.0] で置き換えた

Tensor("mul:0", shape=(1,), dtype=float32)
Tensor("mul_1:0", shape=(1,), dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


array([6.], dtype=float32)

define and run は、通常のプログラミングとメンタルモデルが大きく異なります。
print デバッグもできないし、普通の if 文, for 文も使えなくなります。（ `tf.cond` や `tf.while` を代わりに使います。）

そのため、最近では pytorch や chainer のような define by run のフレームワークの方が人気になりつつあると思います。

一方で、tensorflow はそのような事情を補って余りあるほどの巨大なエコシステムを備えています。
たとえば

- tensorboard 
  - 学習状況の可視化
- ml engine
  - GCP のサービス。マネージドな環境で、tensorflow の学習ができ、HTTP サーバとしてデプロイをすることもできる。
- tensorflow lite
  - モバイルなどのエッジデバイス上で推論を走らせるためのフレームワーク

といったツールやサービスが提供されていますし、有名なニューラルネットワークであれば大抵実装が公開されているというのも重要な利点です。

#### 余談

tensorflow は eager mode というのを最近取り入れだしました。最初に `tf.enable_eager_execution()` を実行しておくと、tensorflow が pytorch や chainer のように動かせる機能です。
先日 tensorflow 2.0 の予定が発表されましたが、その main features の一つにこの eager mode が挙げられていたので、今後もサポートが強化されていきそうです。


#### モデル例

↓に tensorflow と keras で簡単なモデル（計算グラフ）を定義する例を書きます。

In [4]:
import tensorflow as tf

input = tf.placeholder(tf.float32, shape=(None, 28, 28, 1)) # 28x28x1 の行列が入力
x = tf.layers.conv2d(input, filters=10, kernel_size=5)
x = tf.nn.relu(x)
x = tf.layers.max_pooling2d(x, pool_size=2, strides=2)
x = tf.layers.conv2d(x, filters=20, kernel_size=5)
x = tf.nn.relu(x)
x = tf.layers.max_pooling2d(x, pool_size=2, strides=2)
x = tf.reshape(x, [-1, 4 * 4 * 20])
x = tf.layers.dense(x, 50)
x = tf.nn.relu(x)
x = tf.layers.dense(x, 10)
x = tf.nn.softmax(x)
x

<tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float32>

In [5]:
from tensorflow.python.keras.layers import Input, Conv2D, Activation, MaxPooling2D, Dense, Flatten

input = Input(shape=(28, 28, 1))
x = Conv2D(10, kernel_size=5)(input)
x = Activation('relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(20, kernel_size=5)(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)
x = Dense(50)(x)
x = Activation('relu')(x)
x = Dense(10)(x)
x = Activation('softmax')(x)

tf.keras.Model(input, x).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 10)        260       
_________________________________________________________________
activation (Activation)      (None, 24, 24, 10)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 20)          5020      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 20)          0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 20)          0         
__________

### 一旦まとめ

だいたいどのフレームワークでも、

- 入力となる行列データを定義
- convolution などの layer を通す
- 出力の形式を欲しい形式に揃える

というながれになっています。

なので、各フレームワークの layers に関するドキュメントをざっと眺めると、なにができるのかなんとなく把握できると思います。
典型的な問題であれば、ただただ layers を重ねるだけでモデルを作ることができます。
https://keras.io/layers/about-keras-layers/ 


実際に学習をさせるためには、損失関数を定義する必要があるので、損失関数に関するドキュメントも読んでみることをおすすめします。
https://keras.io/ja/objectives/